In [1]:
# Start hierarchical single link clustering by Jaccard distance

In [2]:
def char_gram(sequence, k):
    grams = set()
    
    for i in range(len(sequence) - k + 1):
        grams.add(sequence[i:i+k])
        
    return grams


def create_grams(sequences, k):
    grams = {}
    
    for uri in sequences.keys():
        gram = char_gram(sequences[uri], k)
        grams[uri] = gram
    
    return grams


def JD(s1, s2):
    '''
    # below was too slow
    intersection = len(s1 & s2)
    union = len(s1 | s2)
    '''
    intersection = 0
    union = len(s1)
    
    for gram in s2:
        if gram in s1:
            intersection += 1
        else:
            union +=1
            
    if union == 0:
        return 1
    
    return 1 - (intersection / union)
            


grams = create_grams(sequences, 3)

NameError: name 'sequences' is not defined

In [3]:
def write_grams(grams):
    f = open('grams.txt', 'w')

    for uri in grams:
        f.write(uri + ' ' + ' '.join(grams[uri]) + '\n')

write_grams(grams)

NameError: name 'grams' is not defined

In [4]:
memo = None


def get_JD_memo(key_a, key_b):
    smaller_key = key_a if key_a < key_b else key_b
    larger_key = key_b if key_a < key_b else key_a
    return memo[smaller_key][larger_key]


def create_JD_memo(grams):
    global memo
    
    memo = {}
    
    for key_a in grams:
        for key_b in grams:
            distance = JD(grams[key_a], grams[key_b])
            
            smaller_key = key_a if key_a < key_b else key_b
            larger_key = key_b if key_a < key_b else key_a
                
            if smaller_key not in memo:
                memo[smaller_key] = {larger_key: distance}
            else:
                memo[smaller_key][larger_key] = distance
    

def cluster_indicies(clusters):
    indicies = set()
    
    for key in clusters:
        indicies.add(clusters[key])
        
    return indicies


def cluster_at_index(i, clusters, grams):
    i_grams = {}
    
    for key, value in clusters.items():
        if value == i:
            i_grams[key] = grams[key]
    
    return i_grams
    
    
def closest(grams, clusters, cluster_distance):
    indicies = cluster_indicies(clusters)
    
    clusters_by_index = {}
    for index in indicies:
        clusters_by_index[index] = cluster_at_index(index, clusters, grams)
    
    _min = float('+inf')
    
    for i in indicies:
        for j in indicies:
            if i != j:
                i_cluster = clusters_by_index[i]
                j_cluster = clusters_by_index[j]
                d = cluster_distance(i_cluster, j_cluster)
                
                if d < _min:
                    _min = d
                    most_close = (i, j)
    
    return most_close
    

def hierarchical_cluster(grams, k, cluster_distance):
    print('creating JD memo')
    create_JD_memo(grams)
    
    clusters = {}
    
    for index, key in enumerate(grams.keys()):
        clusters[key] = index
    
    while len(cluster_indicies(clusters)) > k:
        a, b = closest(grams, clusters, cluster_distance)
        
        for key, value in clusters.items():
            if value == a or value == b:
                clusters[key] = min(a, b)
        
        print('merged: ' + str(a) + ' ' + str(b))
        
    return clusters


def shortest_link(cluster_a, cluster_b):
    d = float('+inf')
    
    for key_a in cluster_a:
        for key_b in cluster_b:
            similarity = get_JD_memo(key_a, key_b)
            d = min(d, similarity)
            
    return d

In [5]:
import matplotlib.pyplot as plt
import numpy as np

def show_cdf(results):
    x = sorted(results)
    y = np.arange(1, len(x)+1) / len(x)

    _ = plt.plot(x, y, marker='.', linestyle='none')
    _ = plt.title('Jaccard distance cdf with ' + str(len(results)) + ' points')
    _ = plt.xlabel('Jaccard distance')
    _ = plt.ylabel('fraction of experiments that had a lower Jaccard distance')
    plt.margins(0.02) 
    

def flatten(memo):
    results = []
    
    for key_a in memo:
        for key_b in memo[key_a]:
            results.append(memo[key_a][key_b])
            
    return results

In [6]:
def create_n_grams(grams, n):
    mini_grams = {}
    keys = list(grams.keys())[0:n]
    
    for key in keys:
        mini_grams[key] = grams[key]
    
    return mini_grams


def run_clustering_experiment(n):
    print('total grams: ' + str(len(grams)))
    mini_grams = create_n_grams(grams, n)
    clusters = hierarchical_cluster(mini_grams, 10, shortest_link)
    print(cluster_indicies(clusters))
    
    
mini_grams = create_n_grams(grams, 3)
for gram in mini_grams.items():
    print(gram)
    
create_JD_memo(create_n_grams(grams, 1000))
show_cdf(flatten(memo))

NameError: name 'grams' is not defined

In [7]:
import cProfile
import pstats

cProfile.run('run_clustering_experiment(400)', 'stats')
stats = pstats.Stats('stats')
stats.sort_stats('time').print_stats()


NameError: name 'grams' is not defined

In [8]:
# sort distances, merge in order, union the clusters -> same as single link, but n^2log*n using disjoint set
# incrementally store the distances, ex. go up by .1/some cutoff point
# index grams by short 2^16
# plot JD cdf from just 0 to .1 to determine JD cutoff point
# use role and type to break up dataset
# choose k depending on JD cdf
# times:
# 100 -> 2.422, 200 -> 22.384, 400 -> 188, 800 -> 1367, n^3